In [1]:
import pandas as pd
import os

In [2]:
# Read and concatenate the sales data
directory = "./Data/Sales"
all_files = os.listdir(directory)
sales_df = pd.read_csv(os.path.join(directory, all_files[0])) # Use the first file to form the base dataframe
for file_path in all_files[1:]:
    file_df = pd.read_csv(os.path.join(directory, all_files[0]))
    sales_df = sales_df.append(file_df, ignore_index=True)
sales_df.head()

,identifier,Network,Region,Date,Product,Amount
0,27729554427,'Network 1','Region 1','12-Mar-2016','Product 1',1000.0
1,27722342551,'Network 2','Region 1','16-Mar-2016','Product 1',1122.0
2,27725544272,'Network 3','Region 2','17-Mar-2016','Product 2',2084.0
3,27725326345,'Network 3','Region 2','18-Mar-2016','Product 2',3098.0
4,27729234533,'Network 2','Region 1','01-Apr-2016','Product 1',5671.0


In [3]:
# Enrich sales data with region description
region_df = pd.read_csv("./Data/Region/region_2016.csv")
enriched_df = pd.merge(sales_df, region_df[['Region', 'RegionDescription']], on='Region')
enriched_df.head()

,identifier,Network,Region,Date,Product,Amount,RegionDescription
0,27729554427,'Network 1','Region 1','12-Mar-2016','Product 1',1000.0,Southern Region
1,27722342551,'Network 2','Region 1','16-Mar-2016','Product 1',1122.0,Southern Region
2,27729234533,'Network 2','Region 1','01-Apr-2016','Product 1',5671.0,Southern Region
3,27725678534,'Network 2','Region 1','15-Apr-2016','Product 3',1747.0,Southern Region
4,27729554427,'Network 1','Region 1','16-Apr-2016','Product 2',1801.0,Southern Region


In [4]:
# write output files providing total sales amount and number of sales per region and network to the output location
amount_grouped_df = enriched_df.groupby(by=["Region", "Network"], as_index=False)[['Amount']].sum()
amount_grouped_df.head()

,Region,Network,Amount
0,'Region 1','Network 1',5602.0
1,'Region 1','Network 2',17080.0
2,'Region 2','Network 3',14220.0


In [5]:
counts_grouped_df = enriched_df.groupby(by=["Region", "Network"], as_index=False)[['identifier']].count()
counts_grouped_df.rename(columns={'identifier':'Total Sales Number'}, inplace=True)
counts_grouped_df.head()

,Region,Network,Total Sales Number
0,'Region 1','Network 1',4
1,'Region 1','Network 2',6
2,'Region 2','Network 3',6


In [6]:
summary_df = pd.merge(amount_grouped_df, counts_grouped_df, on=['Region', 'Network'])
summary_df.head()

,Region,Network,Amount,Total Sales Number
0,'Region 1','Network 1',5602.0,4
1,'Region 1','Network 2',17080.0,6
2,'Region 2','Network 3',14220.0,6


In [8]:
import time
file_name = f"./Data/Output/processed_data_{time.time()}.csv"
summary_df.to_csv(file_name, index=False)